In [ ]:
import rasterio
import affine
from rasterio import plot
import matplotlib.pyplot as plt
import shapefile
from pyproj import CRS, transform, Transformer
import os
%matplotlib inline

In [ ]:
IMAGE_PIXEL = 15
IMAGE_WIDTH = 732
LIST_SHAPEFILES = ["../Données/heigvd/central_highlands_2_other/central_highlands_2_other.shp",
                  "../Données/heigvd/central_highlands_2_test/central_highlands_2_test.shp",
                  "../Données/heigvd/central_higlands_1_other/central_higlands_1_other.shp"]
CORRESPONDANCES = {"1":"cacao", "2":"coffee","3":"complex_oil", "4":"nativevege", "5":"oil_palm", "6":"rubber", "7":"unknown", 
                 "8":"seasonal", "9":"urban", "10":"water", "11":"other_tree", "12":"other_no_tree", "13":"native_no_tree",
                 "14":"water_other", "15":"pepper", "16":"cassava", "17":"tea", "18":"rice", "19":"banana", "20":"baby_palm", 
                 "21":"cur_off-regrow", "22":"natural_wetland", "23":"intercrop", "24":"deciduous_forest", "25":"stick_pepper", 
                 "26":"flooded_plantation", "27":"pine_trees", "28":"coconut", "29":"banboo", "30":"savana", "31":"mango", 
                 "32":"other_fruit_tree_crop", "33":"water_mine", "0":"not_labeled", "-1":"ambiguous"}
SHAPEFILE_ESPG = 4326

In [ ]:
points = []
for path in LIST_SHAPEFILES:
    sf = shapefile.Reader(path)
    shapes = sf.shapes()
    for point in sf.records():
        points.append((shapes[point.oid].points[0], point[0]))
        
def get_labels(begin, end):
    """Donne tous les labels contenu entre 2 positions géographiques"""
    labels = set()
    long1, lat1 = convert_to_format(begin)
    long2, lat2 = convert_to_format(end)
    for point in points:
        if long1 <= point[0][0] <= long2 and lat2 <= point[0][1] <= lat1:
            labels.add(point[1])
    return labels

In [ ]:
imagePath = '../Données/S2A_MSIL2A_20190227T030651_N0211_R075_T48PZV_20190227T083207/S2A_MSIL2A_20190227T030651_N0211_R075_T48PZV_20190227T083207.SAFE/GRANULE/L2A_T48PZV_A019234_20190227T031435/IMG_DATA/R10m/T48PZV_20190227T030651_'
band2 = rasterio.open(imagePath+'B02_10m.jp2', driver='JP2OpenJPEG') #blue
band3 = rasterio.open(imagePath+'B03_10m.jp2', driver='JP2OpenJPEG') #green
band4 = rasterio.open(imagePath+'B04_10m.jp2', driver='JP2OpenJPEG') #red
band8 = rasterio.open(imagePath+'B08_10m.jp2', driver='JP2OpenJPEG') #near impact

In [ ]:
transformer = Transformer.from_crs(band2.gcps[1], SHAPEFILE_ESPG)
def convert_to_format(coords):
    """Sert à convertir le format de géolocalisation de l'image pour matcher avec celui des points des shapefiles"""
    conversion = transformer.transform(coords[0], coords[1])
    return (conversion[1], conversion[0])

In [ ]:
def split_band(band):
    """Split une bande de Sentinel-2 en bande plus petite
    IMAGE_WIDTH^2 images de IMAGE_PIXELxIMAGE_PIXEL"""
    result = []
    my_band = band.read(1)
    for x in range(IMAGE_WIDTH):
        columns = []
        for y in range(IMAGE_WIDTH):
            lines = []
            for i in range(IMAGE_PIXEL):
                cells = []
                for j in range(IMAGE_PIXEL):
                    cells.append(my_band[i + IMAGE_PIXEL * x][j + IMAGE_PIXEL * y])
                lines.append(cells)
            columns.append(lines)
        result.append(columns)
    return result

In [ ]:
def create_smaller_images(blue, green, red, infra_red, path_name, file_name):
    """Créée des images plus petites pouvant être mieux traitées par le réseau de neurones et les classes dans le bon dossier
    selon s'il existe des positions déjà labelée dans le dossier (ambigious si plusieurs, not_labeled si pas de coordonnées)"""
    smaller_blue = split_band(blue)
    print("Fin bande bleue")
    smaller_green = split_band(green)
    print("Fin bande verte")
    smaller_red = split_band(red)
    print("Fin bande rouge")
    smaller_infra_red = split_band(infra_red)
    print("Fin bande infra rouge")
    if not os.path.isdir(path_name):
        os.mkdir(path_name)
    for _, directory in CORRESPONDANCES.items():
        if not os.path.isdir(path_name + "/" + directory):
            os.mkdir(path_name + "/" + directory)
    for x in range(IMAGE_WIDTH):
        for y in range(IMAGE_WIDTH):
            begin = blue.transform * (y * IMAGE_PIXEL,x * IMAGE_PIXEL)
            end = blue.transform * ((y + 1) * IMAGE_PIXEL,(x + 1) * IMAGE_PIXEL)   
            transform = affine.Affine(10.0, 0, begin[0], 0.0, -10.0, begin[1])
            #Trie selon s'il y a des points déjà répertorié dans le fichier.
            labels = get_labels(begin, end)
            image_name = path_name + "/"
            if len(labels) == 0:
                if not os.path.exists(path_name + "/" + CORRESPONDANCES['0'] + "/" + str(x)):
                    os.mkdir(path_name + "/" + CORRESPONDANCES['0'] + "/" + str(x))
                image_name += CORRESPONDANCES['0'] + "/" + str(x) + "/"
            elif len(labels) > 1:
                image_name += CORRESPONDANCES['-1'] + "/"
            else:
                image_name += CORRESPONDANCES[str(next(iter(labels)))] + "/"
            image_name += file_name
            smaller_image = rasterio.open(image_name + '_' + str(x) + '_' + str(y) \
                                              + '.tiff', 'w', driver='Gtiff', 
                          width=IMAGE_PIXEL, height=IMAGE_PIXEL, count=4, crs=blue.crs, 
                          transform=transform, 
                          dtype=blue.dtypes[0]
                         )
            smaller_image.write(smaller_blue[x][y], 4)
            smaller_image.write(smaller_green[x][y], 3)
            smaller_image.write(smaller_red[x][y], 2)
            smaller_image.write(smaller_infra_red[x][y], 1)
            smaller_image.close()
    print("Fichiers tiff créés.")

In [ ]:
create_smaller_images(band2, band3, band4, band8, "output/milieu", "milieu")